# Exploring the RADCLIM Median Zarr archive

### Abstract

This notebook describes how to find the analogue dates and the median field for a given date between the 1st January 1940 and the 31st December 2016 in the **RADCLIM-Analogs-median** dataset. 

This dataset provides maps of the analogues median at 1 km resolution of the hourly accumulated precipitation over Belgium from 1940 to 2016. We used the analog technique to provide for every day in the past (1940 – 2016) the 25 best analogs selected from the high resolution RMI RADCLIM radar database that is available from 2017 to 2022. The median was then extracted from this ensemble.

This notebook is provided as supplementary material for the article:

* Debrie, E., Demaeyer, J., and Vannitsem, S.: Hourly precipitation series over Belgium based on the Analogue Technique, Earth Syst. Sci. Data Discuss. [preprint], doi:, in review, 2025.

Before running this notebook, the dataset must first be downloaded from the Zenodo platform (https://doi.org/10.5281/zenodo.14965710) and installed following the installation instruction available there.





### Goal of the present notebook

In the present notebook, we are going to look for the analogue dates for a given date, and then plot the median precipitation field extracted from the RADCLIM data.


### Code

Imports

In [ ]:
import os

In [ ]:
import xarray as xr

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

import cmocean.cm as cm

import cartopy

import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
import datetime

Some parameter variables:

In [ ]:
path_to_zarr = "./" # Where you unpacked the dataset.
fields_zarr_archive = "radclim-analogs-median.zarr"
analogs_zarr_archive = "precipitation-analogs.zarr"

Loading the RADCLIM Median Zarr archive

In [ ]:
ds = xr.open_zarr(os.path.join(path_to_zarr, fields_zarr_archive))

In [ ]:
ds

Loading the Analogues database

In [ ]:
analogs_ds = xr.open_zarr(os.path.join(path_to_zarr, analogs_zarr_archive))

In [ ]:
analogs_ds

### Usage example

Determining the analogues of the 14th of July 2010, and plotting the median field:

In [ ]:
analogs = analogs_ds.sel(day='2010-07-14')

In [ ]:
analogs.analog_dates.to_numpy()

Plotting daily accumulation (sum) of the median of the analogues for that day:

In [ ]:
proj = ccrs.PlateCarree()
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': proj},figsize=(14,8),)
isodate = '2010-07-14'
levels = np.linspace(0., 1., 11) 

#  finding all the hourly fields for the 14 July 2020
almostaday = np.timedelta64(23, 'h') + np.timedelta64(59, 'm')
dss = ds.sel(time=slice(np.datetime64(isodate), np.datetime64(isodate)+almostaday))

# plotting the sum
im = ax.contourf(dss.longitude, dss.latitude, dss.tp_median.sum(dim='time'), cmap=cm.rain, 
                 levels=levels, extend='max')
cbar = fig.colorbar(im, ax=ax, ticks=levels, label='Precipitation Median [mm]')

time_str = str(dss.time[0].to_numpy()).split('T')[0]
ax.set_title('Date: '+time_str)
ax.set_extent([2.5, 6.5, 49., 52.])
ax.coastlines(resolution='50m')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels={"bottom": "x", "left": "y"}, );


The end.